In [ ]:
import os
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from youtube_transcript_api.formatters import SRTFormatter
from urllib.parse import urlparse, parse_qs
import gradio as gr
import requests
from openai import OpenAI


d:\AI\LLMENG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ollama_url = "http://localhost:11434/v1"

ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [3]:
requests.get("http://localhost:11434/").content


b'Ollama is running'

In [4]:
system_prompt = "You are a YouTube video summarizer.\
      Your task is to analyze the provided video transcript and generate a concise,\
          clear summary of the main points and key ideas. Focus on the essential information, \
            remove any irrelevant details, and make sure the summary is easy to understand and digest.\
                  Keep the summary brief while maintaining the core message of the video"

In [5]:

def user_prompt(transcript):
    return  f"Please summarize the following video transcript: {transcript} \
    Give a concise summary highlighting the key points and major takeaways from the video."

In [6]:
def messages_for(web):
    return[
        {'role':"system", "content":system_prompt},
        {'role':"user", "content":user_prompt(web)}
    ]

In [ ]:


def video_summarizer(url: str):
    try:
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        video_id = query_params.get('v', [None])[0]

        if not video_id:
            return "Invalid YouTube URL"

        tran = YouTubeTranscriptApi.fetch(video_id)
        transcript_text = " ".join([snip.text for snip in tran])
        return ai_summary(transcript_text)

    except (TranscriptsDisabled, NoTranscriptFound):
        return "Transcript not available for this video"
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:

 

def ai_summary(content):  
    print(content)  
    mess = messages_for(content)
    response = ollama.chat.completions.create(model="llama3.2", messages= mess)
    r =response.choices[0].message.content
    return r


    
   

In [ ]:
message_input = gr.Textbox(label="Youtube Video Link", info="Enter a video link",lines = 8)
message_output = gr.Textbox(label="Summary of YouTube Video",lines = 15)

# Define the Gradio interface
view = gr.Interface(
    fn=video_summarizer,           # The function to run
    title="YouTube Video Summarizer",   # Interface title
    inputs=message_input,         
    outputs=message_output,        
    examples=['https://www.youtube.com/watch?v=WnP6jDvupiY'],  # Example video URL
    flagging_mode="never"          # Disable flagging
)


# Launch the interface
view.launch()




* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


['what is data', 'brought to you by the university of', 'houston libraries', 'we hear the word data all the time', 'it is collected analyzed shared hacked', 'bought and sold', 'but what exactly is data', 'data are collected observations or', 'measurements represented as text numbers', 'or multimedia', 'when we think of data we may immediately', 'think of numbers', 'but data can also be field notes', 'videos', 'audio recordings', 'photographs', 'documents and transcripts', 'what you consider to be data will be', 'different depending on your area of', 'research', 'for example', "if you're researching the impact of", 'wildfires on national parks your data', 'may be satellite maps and images', "but if you're researching graduation", 'rates of college students with faculty', 'mentors', 'your data might be the number of', 'graduates each year and the amount of', 'time taken to earn their degree', 'data will be different based on what you', 'study', 'data can be broadly categorized as', 'qual

: 